In [3]:
%pip install matplotlib
%pip install seaborn
%matplotlib inline

import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

     ---------------------------------------- 0.0/7.6 MB ? eta -:--:--
     ---------------------------------------- 0.0/7.6 MB ? eta -:--:--
     ---------------------------------------- 0.0/7.6 MB ? eta -:--:--
     ---------------------------------------- 0.0/7.6 MB ? eta -:--:--
     ---------------------------------------- 0.0/7.6 MB ? eta -:--:--
     ---------------------------------------- 0.0/7.6 MB ? eta -:--:--
     ---------------------------------------- 0.0/7.6 MB ? eta -:--:--
     ---------------------------------------- 0.0/7.6 MB ? eta -:--:--
     ---------------------------------------- 0.0/7.6 MB ? eta -:--:--
     ---------------------------------------- 0.0/7.6 MB ? eta -:--:--
     ---------------------------------------- 0.0/7.6 MB ? eta -:--:--
     ---------------------------------------- 0.0/7.6 MB ? eta -:--:--
     ---------------------------------------- 0.0/7.6 MB 59.5 kB/s eta 0:02:08
     ---------------------------------------- 0.0/7.6 MB 59.5 kB/s et

In [5]:
url = 'https://raw.githubusercontent.com/SamapanThongmee/SET_Index_Options_Premium_Analysis/main/SET50_Index_Options_Premium.csv'
SET50_Options_Premium = pd.read_csv(url)
SET50_Options_Premium['Date']= pd.to_datetime(SET50_Options_Premium['Date'])
SET50_Options_Premium

,Date,Symbol,Open,High,Low,Close,SP,Vol,OI
0,2006-04-28,S50M06,536.5,542.0,535.6,536.2,536.0,152,98
1,2006-05-02,S50M06,536.0,537.2,536.0,536.1,536.1,53,114
2,2006-05-03,S50M06,537.0,538.5,537.0,538.5,538.5,75,132
3,2006-05-04,S50M06,536.8,536.8,535.6,535.7,535.7,81,166
4,2006-05-08,S50M06,536.5,542.0,536.5,541.0,541.0,109,165
...,...,...,...,...,...,...,...,...,...
16906,2023-11-24,S50Z23,868.0,872.0,863.0,864.0,864.0,178702,541068
16907,2023-11-27,S50Z23,862.0,864.0,856.0,861.0,861.0,199182,561082
16908,2023-11-28,S50Z23,864.0,870.0,862.0,868.0,868.0,186742,583118
16909,2023-11-29,S50Z23,871.0,872.0,858.0,858.0,858.0,238226,592627
